In [ ]:
import builtwith
import whois
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

### Página web

In [ ]:
URL = 'https://tarifaluzhora.es/'

### Evaluación inicial

In [ ]:
# Check robots.txt
url_robots = URL + '/robots.txt'
page = requests.get(url_robots)
print(page.text)

Observamos en el archivo robots.txt que para cualquier rastreador se habilitan todos los elementos gráficos, código y estilos y lo que se deshabilita son la parte para registro de usuarios y configuraciones de la web.

In [ ]:
# Tecnologia del Sitio
print(builtwith.builtwith(URL))

In [ ]:
# Propietario del Sitio
print(whois.whois(URL))

### Scraping por fecha

In [ ]:
def scrap_data(tipo_tarifa,d,m,y):
    
    url = 'https://tarifaluzhora.es/?tarifa=' + tipo_tarifa + '&fecha=' + d + '%2F' + m + '%2F' + y

    # Obtenemos página
    raw_html = requests.get(url).text

    # La parseamos
    data = BeautifulSoup(raw_html, 'html.parser')
    
    # Generamos dataframe mientras buscamos datos y los limpiamos
    df = pd.DataFrame()

    df['Fecha'] = [data.select("span")[6].string]
    df['Franja más cara'] = [data.select("span")[9].string]
    df['Franja más barata'] = [data.select("span")[7].string]

    tags_hora=data.find_all(itemprop="description")    
    tags_precio=data.find_all(itemprop="price")

    a=zip(tags_hora, tags_precio)

    for i,j in a:
        df[i.string] = [j.string.replace(' €/kWh','')]
    
    return df


# Tarifas: normal, discriminacion (discriminacion horaria), coche_electrico (vehiculos electricos)
tarifa = 'normal'

# Creamos dataset
dataset = pd.DataFrame()

# Obtenemos los datos de Enero
for i in range (1,32):
    dataset =pd.concat([dataset, scrap_data(tarifa, str(i).zfill(2), '01', '2021')])

# Obtenemos los datos de Febrero
for i in range (1,29):
    dataset =pd.concat([dataset, scrap_data(tarifa, str(i).zfill(2), '02', '2021')])

# Obtenemos los datos de Marzo
for i in range (1,32):
    dataset =pd.concat([dataset, scrap_data(tarifa, str(i).zfill(2), '03', '2021')])

In [ ]:
# dataset

In [ ]:
dataset.to_csv(r'tarifa_electrica_EneMar2021.csv',index=False)